<a href="https://colab.research.google.com/github/Max1897/ECE7363_project/blob/main/src/ECE7363_TCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://colab.research.google.com/drive/1MVHhaOw2Xro3RMN4ApDCDo7R9taLCRYJ#scrollTo=1mbvfJAcZCGY

In [ ]:
!pip install 'u8darts[all]'

In [55]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import TCNModel

from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from datetime import datetime

import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")


%matplotlib inline

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

In [5]:
from google.colab import drive
drive.mount('/content/drive')
work_dir = "/content/drive/MyDrive/Colab Notebooks/data/Network_proj/TCN/"

Mounted at /content/drive


#**Prepare data**

In [6]:
def change_granularity(data, addup = 1):
  data_length = data.shape[0]
  Temp = np.sum(data[0:addup],axis = 0)
  itr_end = data_length // addup
  for i in tqdm(range(1,itr_end)):
    temp0 = np.sum(data[i*addup:i*addup+addup],axis = 0)
    Temp = np.vstack((Temp,temp0))
  
  return Temp

In [7]:
#Read files
train_data = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/data/Network_proj/" + "GEANT_train.txt",dtype = float)
test_data = np.loadtxt("/content/drive/MyDrive/Colab Notebooks/data/Network_proj/" + "GEANT_test.txt",dtype = float)

In [8]:
num_nodes = 22
ind = []
for i in range(num_nodes):
  for j in range(num_nodes):
    if i == j :ind.append(i*num_nodes + j)

mask = np.ones((num_nodes ** 2), dtype = bool)
mask[[ind]] = False
Train_data = train_data[:,mask]
Test_data = test_data[:,mask]

In [9]:
gran_addup = 2
Train_data = change_granularity(Train_data,gran_addup)
Test_data = change_granularity(Test_data,gran_addup)

  0%|          | 0/4039 [00:00<?, ?it/s]

  0%|          | 0/1345 [00:00<?, ?it/s]

In [10]:
All = np.vstack((Train_data,Test_data))

Making each demand pair a TimeSeries

In [11]:
SERIES = []
for d in All.T:
  df = pd.DataFrame(d)
  series = TimeSeries.from_dataframe(df)
  SERIES.append(series)

#**Pre - Processing**

scaling

In [12]:
SCALERS = []
SCALED_SERIES = []
for ts in SERIES:
  scaler = Scaler()
  series_scaled = scaler.fit_transform(ts)
  SCALERS.append(scaler)
  SCALED_SERIES.append(series_scaled)

train, test split

In [13]:
TRAIN_SERIES = []
TEST_SERIES = []
for ts in SCALED_SERIES:
  train_series = ts[:4400]
  test_series = ts[4400:]
  TRAIN_SERIES.append(train_series)
  TEST_SERIES.append(test_series)

#**Model Setup**

In [14]:
MODEL = TCNModel(
    input_chunk_length=10,
    output_chunk_length=1, 
    n_epochs=15, 
    dropout=0.2, 
    dilation_base=2, 
    weight_norm=True,
    kernel_size=3,
    num_filters=1,
    random_state=0,
    save_checkpoints = True,
    pl_trainer_kwargs={
      "accelerator": "gpu",
      "devices": [0]
    }
)

#**Model Training**
Estimated 2h

In [15]:
# MODEL.fit(TRAIN_SERIES,verbose = True)
# MODEL.save(work_dir + "TCN.pkl")

#**Prediction**

In [22]:
MODEL = TCNModel.load(work_dir + "TCN.pkl")

In [49]:
def random_data(TEST_SERIES):
  r = np.random.randint(10,986)
  sequences = []
  data_point = []
  for ts in TEST_SERIES:
    sequences.append(ts[r-10:r])
    data_point.append(ts[r])
  return data_point, sequences

# def inverse_scale()

In [82]:
data_point, sequences = random_data(TEST_SERIES)

pred = MODEL.predict(n = 1,series = sequences)

Predicting: 0it [00:00, ?it/s]

inverse transform

In [83]:
pred_inv = []
true_data = []
for i in range(462):
  p = SCALERS[i].inverse_transform(pred[i])
  t = SCALERS[i].inverse_transform(data_point[i])
  pred_inv.append(p)
  true_data.append(t)


#**Output to AMPL**

In [71]:
def output_to_AMPL(demands,dir):
  
  demand_matrix = []
  for i in range(1,23):
    line0 = repr(i) 
    for j in range(1,23):
      if i == j: continue
      line1 = line0 + " " + repr(j)
      demand_matrix.append(line1)


  f = open(dir,'w')
  for i,d in enumerate(demands):
    d = np.squeeze(d.values())
    d = np.ndarray.tolist(d)
    if d < 0: d = 0.0
    demand_matrix[i] = repr(i+1) + " "+ demand_matrix[i] + " " + repr(d) + "\n"
    f.write(demand_matrix[i])
  f.close()

  return demand_matrix

In [84]:
now = datetime.now()
date_time = now.strftime("%d%H%M%S")
output_dir = "/content/drive/MyDrive/Colab Notebooks/data/Network_proj/AMPL demands/"
pred_dir = output_dir + date_time + "TCN_pred.txt"
true_dir = output_dir + date_time + "TCN_true.txt"
demand_matrix_pred = output_to_AMPL(pred_inv,pred_dir)
demand_matrix_true = output_to_AMPL(true_data,true_dir)

#**Script**

In [70]:
l = np.squeeze(pred_inv[0].values())
np.ndarray.tolist(l)

61.023843244984285